[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/qubvel/segmentation_models.pytorch/blob/main/examples/save_load_model_and_share_with_hf_hub.ipynb)

In [1]:
import segmentation_models_pytorch as smp

## Save to local directory and load back

In [2]:
model = smp.Unet()

# save the model
model.save_pretrained("saved-model-dir/unet/")

# load the model
restored_model = smp.from_pretrained("saved-model-dir/unet/")

Loading weights from local directory


## Save model with additional metadata

In [3]:
model = smp.Unet()

# save the model
model.save_pretrained(
    "saved-model-dir/unet-with-metadata/",
    # additional information to be saved with the model
    # only "dataset" and "metrics" are supported
    dataset="PASCAL VOC",  # only string name is supported
    metrics={  # should be a dictionary with metric name as key and metric value as value
        "mIoU": 0.95,
        "accuracy": 0.96,
    },
)

In [4]:
!cat "saved-model-dir/unet-with-metadata/README.md"

---
library_name: segmentation-models-pytorch
license: mit
pipeline_tag: image-segmentation
tags:
- model_hub_mixin
- pytorch_model_hub_mixin
- segmentation-models-pytorch
- semantic-segmentation
- pytorch
languages:
- python
---
# Unet Model Card

Table of Contents:
- [Load trained model](#load-trained-model)
- [Model init parameters](#model-init-parameters)
- [Model metrics](#model-metrics)
- [Dataset](#dataset)

## Load trained model
```python
import segmentation_models_pytorch as smp

model = smp.from_pretrained("<save-directory-or-this-repo>")
```

## Model init parameters
```python
model_init_params = {
    "encoder_name": "resnet34",
    "encoder_depth": 5,
    "encoder_weights": "imagenet",
    "decoder_use_batchnorm": True,
    "decoder_channels": (256, 128, 64, 32, 16),
    "decoder_attention_type": None,
    "in_channels": 3,
    "classes": 1,
    "activation": None,
    "aux_params": None
}
```

## Model metrics
```json
{
    "mIoU": 0.95,
    "accuracy": 0.96
}
```

## Dat

## Share model with HF Hub

In [5]:
from huggingface_hub import notebook_login

# You only need to run this once on the machine,
# the token will be stored for later use
notebook_login()

In [6]:
model = smp.Unet()

# save the model and share it on the HF Hub (https://huggingface.co/models)
model.save_pretrained(
    "qubvel-hf/unet-with-metadata/",
    push_to_hub=True,  # <----------    push the model to the hub
    private=False,  # <----------    make the model private or or public
    dataset="PASCAL VOC",
    metrics={"mIoU": 0.95, "accuracy": 0.96},
)

# see result here https://huggingface.co/qubvel-hf/unet-with-metadata

model.safetensors:   0%|          | 0.00/97.8M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/qubvel-hf/unet-with-metadata/commit/4ac3d2925d34cf183dc79a2e21b6e2f4bfe87586', commit_message='Push model using huggingface_hub.', commit_description='', oid='4ac3d2925d34cf183dc79a2e21b6e2f4bfe87586', pr_url=None, pr_revision=None, pr_num=None)

## Save model with preprocessing (using albumentations)

In [ ]:
!pip install -U albumentations numpy==1.*

In [2]:
import albumentations as A
import segmentation_models_pytorch as smp

In [3]:
# define a preprocessing transform for image that would be used during inference
preprocessing_transform = A.Compose([A.Resize(256, 256), A.Normalize()])

model = smp.Unet()

In [4]:
directory_or_repo_on_the_hub = "qubvel-hf/unet-with-transform"

# save the model
model.save_pretrained(directory_or_repo_on_the_hub, push_to_hub=True)

# save transform
preprocessing_transform.save_pretrained(directory_or_repo_on_the_hub, push_to_hub=True)

model.safetensors:   0%|          | 0.00/97.8M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/qubvel-hf/unet-with-transform/commit/680dad16431fa6efbb25832d33a24056bdf7dc1a', commit_message='Push transform using huggingface_hub.', commit_description='', oid='680dad16431fa6efbb25832d33a24056bdf7dc1a', pr_url=None, pr_revision=None, pr_num=None)

Now, let's restore model and preprocessing transform for inference:

In [5]:
restored_model = smp.from_pretrained(directory_or_repo_on_the_hub)
restored_transform = A.Compose.from_pretrained(directory_or_repo_on_the_hub)

print(restored_transform)

Loading weights from local directory
Compose([
  Resize(p=1.0, height=256, width=256, interpolation=1),
  Normalize(p=1.0, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, normalization='standard'),
], p=1.0, bbox_params=None, keypoint_params=None, additional_targets={}, is_check_shapes=True)


In [6]:
# You can also save training augmentations to the Hub too (and load it back)!
#! Just make sure to provide key="train" when saving and loading the augmentations.

train_augmentations = A.Compose(
    [
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.2),
        A.ShiftScaleRotate(p=0.5),
    ]
)

train_augmentations.save_pretrained(
    directory_or_repo_on_the_hub, key="train", push_to_hub=True
)

restored_train_augmentations = A.Compose.from_pretrained(
    directory_or_repo_on_the_hub, key="train"
)
print(restored_train_augmentations)

Compose([
  HorizontalFlip(p=0.5),
  RandomBrightnessContrast(p=0.2, brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), brightness_by_max=True),
  ShiftScaleRotate(p=0.5, shift_limit_x=(-0.0625, 0.0625), shift_limit_y=(-0.0625, 0.0625), scale_limit=(-0.09999999999999998, 0.10000000000000009), rotate_limit=(-45, 45), interpolation=1, border_mode=4, value=0.0, mask_value=0.0, rotate_method='largest_box'),
], p=1.0, bbox_params=None, keypoint_params=None, additional_targets={}, is_check_shapes=True)


See saved model and `albumentations` configs on the hub: https://huggingface.co/qubvel-hf/unet-with-transform/tree/main